🔄 Data Transformation Requirements
1. Name Processing
Task:
Split the Name column into two separate fields:
First Name
Last Name
Use the space as a separator for splitting.
Additional Processing:
Remove common prefixes:
Examples: Mr., Mrs., Miss, Dr.
Remove common suffixes:
Examples: Jr., Sr., II, III

In [1]:
pip install sqlalchemy pyodbc pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyodbc
import pandas as pd


In [16]:
from sqlalchemy import create_engine
import pandas as pd
import urllib

# Create connection string
params = urllib.parse.quote_plus(
    "DRIVER={SQL Server};SERVER=DESKTOP-FIQ4FDL\\SQLEXPRESS03;DATABASE=sqlpythondb;UID=sa;PWD=manisha123;"
)

# SQLAlchemy engine
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [17]:
# Load customer_id and name into DataFrame
customers = pd.read_sql("SELECT * FROM customers", engine)

print(customers.head())

   customer_id                        email           phone  \
0            1           vayala@example.net  (619) 723-4258   
1            2  taylorcatherine@example.net  (537) 674-1158   
2            3         dsanchez@example.net  (810) 256-4505   
3            4    jessicaknight@example.com  (423) 222-9779   
4            5        qwilliams@example.com  (220) 763-3522   

                                             address registration_date  \
0                        USNS Santiago, FPO AE 80872        2025-01-25   
1  38783 Oliver Street, West Kristenborough, MT 9...        2023-07-13   
2     6845 Steele Turnpike, West Erikabury, UT 37487        2023-08-18   
3         1631 Alexis Meadows, Lake Amanda, CA 75179        2024-12-08   
4  2274 Williams Heights Suite 895, Andersonhaven...        2024-02-03   

  loyalty_status first_name last_name middle_name  
0           Gold   Michelle      Kidd           -  
1         Silver       Brad    Newton           -  
2         Bronze    

In [30]:
# Define common prefixes and suffixes
prefixes = ['Mr.', 'Mrs.', 'Miss', 'Dr.']
suffixes = ['Jr.', 'Sr.', 'II', 'III']

def clean_name(name):
    if pd.isna(name):
        return pd.Series(["", ""])
    
    name = name.strip()

    for prefix in prefixes:
        if name.startswith(prefix + ' '):
            name = name[len(prefix)+1:]
            break
    
    for suffix in suffixes:
        if name.endswith(' ' + suffix):
            name = name[:-(len(suffix)+1)]
            break

    parts = name.strip().split(" ", 1)
    first = parts[0]
    last = parts[1] if len(parts) > 1 else ""
    return pd.Series([first, last])

# Apply transformation
customers[['first_name', 'last_name']] = customers['name'].apply(clean_name)

print(customers[['first_name', 'last_name']])


    first_name last_name
0     Michelle      Kidd
1         Brad    Newton
2        Larry    Torres
3     Kimberly     Price
4      Matthew  Phillips
..         ...       ...
995      Jerry  Mcdaniel
996       Jodi   Simpson
997    Crystal     Brown
998    Gregory    Duarte
999      Jason     Walls

[1000 rows x 2 columns]


In [36]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("ALTER TABLE customers ADD first_name VARCHAR(100), last_name VARCHAR(100)"))

customers = pd.read_sql("SELECT * FROM customers", engine)

print(customers.head())    


   customer_id              name                        email           phone  \
0            1     Michelle Kidd           vayala@example.net  (619) 723-4258   
1            2       Brad Newton  taylorcatherine@example.net  (537) 674-1158   
2            3      Larry Torres         dsanchez@example.net  (810) 256-4505   
3            4    Kimberly Price    jessicaknight@example.com  (423) 222-9779   
4            5  Matthew Phillips        qwilliams@example.com  (220) 763-3522   

                                             address registration_date  \
0                        USNS Santiago, FPO AE 80872        2025-01-25   
1  38783 Oliver Street, West Kristenborough, MT 9...        2023-07-13   
2     6845 Steele Turnpike, West Erikabury, UT 37487        2023-08-18   
3         1631 Alexis Meadows, Lake Amanda, CA 75179        2024-12-08   
4  2274 Williams Heights Suite 895, Andersonhaven...        2024-02-03   

  loyalty_status  
0           Gold  
1         Silver  
2         B

In [37]:
import re

# Clean the 'name' field: remove prefixes and suffixes
def clean_name(name):
    name = re.sub(r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s+', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+(Jr\.|Sr\.|II|III)$', '', name, flags=re.IGNORECASE)
    return name.strip()

# Apply cleaning
customers['clean_name'] = customers['name'].apply(clean_name)

# Split into first and last name
customers[['first_name', 'last_name']] = customers['clean_name'].str.extract(r'^(\S+)\s+(.*)$')


In [40]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE customers
        ADD first_name VARCHAR(100), last_name VARCHAR(100)
    """))


In [44]:
with engine.begin() as conn:
    conn.execute(text("""
        IF COL_LENGTH('customers', 'middle_name') IS NULL
        BEGIN
            ALTER TABLE customers ADD middle_name VARCHAR(100)
        END
    """))


In [56]:
columns_check = pd.read_sql("""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'customers'
""", engine)

print(columns_check)

         COLUMN_NAME
0        customer_id
1              email
2              phone
3            address
4  registration_date
5     loyalty_status
6         first_name
7          last_name
8        middle_name


In [42]:
import re

def split_full_name(name):
    # Remove prefixes and suffixes
    name = re.sub(r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s+', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+(Jr\.|Sr\.|II|III)$', '', name, flags=re.IGNORECASE)
    parts = name.strip().split()

    # Assign name parts
    first = parts[0] if len(parts) > 0 else None
    middle = " ".join(parts[1:-1]) if len(parts) > 2 else None
    last = parts[-1] if len(parts) > 1 else None

    return pd.Series([first, middle, last])


In [48]:
customers[['first_name', 'middle_name', 'last_name']] = customers['name'].apply(split_full_name)


In [49]:
with engine.begin() as conn:
    for _, row in customers.iterrows():
        conn.execute(
            text("""
                UPDATE customers
                SET first_name = :first_name,
                    middle_name = :middle_name,
                    last_name = :last_name
                WHERE customer_id = :customer_id
            """),
            {
                "first_name": row['first_name'],
                "middle_name": row['middle_name'],
                "last_name": row['last_name'],
                "customer_id": row['customer_id']
            }
        )


In [50]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE customers DROP COLUMN name
    """))


In [53]:
import pandas as pd

# Reordered column SQL
query = """
SELECT 
    customer_id,
    first_name,
    middle_name,
    last_name,
    email,
    phone,
    address,
    registration_date,
    loyalty_status
FROM customers
"""

# Execute the query and load into a DataFrame
table= pd.read_sql(query, engine)

# Display the result
table.head()


,customer_id,first_name,middle_name,last_name,email,phone,address,registration_date,loyalty_status
0,1,Michelle,None,Kidd,vayala@example.net,(619) 723-4258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold
1,2,Brad,None,Newton,taylorcatherine@example.net,(537) 674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver
2,3,Larry,None,Torres,dsanchez@example.net,(810) 256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze
3,4,Kimberly,None,Price,jessicaknight@example.com,(423) 222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold
4,5,Matthew,None,Phillips,qwilliams@example.com,(220) 763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold


In [54]:
from sqlalchemy import text

# Update all NULL middle_name values to '-'
with engine.begin() as conn:  # auto-commits on success
    conn.execute(text("""
        UPDATE customers
        SET middle_name = '-'
        WHERE middle_name IS NULL
    """))

print("All NULL middle_name values replaced with '-'")


All NULL middle_name values replaced with '-'


In [12]:
# Reordered column SQL
query = """
SELECT 
    customer_id,
    first_name,
    middle_name,
    last_name,
    email,
    phone,
    address,
    registration_date,
    loyalty_status
FROM customers
"""

# Execute the query and load into a DataFrame
table= pd.read_sql(query, engine)

# Display the result
table.head()

,customer_id,first_name,middle_name,last_name,email,phone,address,registration_date,loyalty_status
0,1,Michelle,-,Kidd,vayala@example.net,(619) 723-4258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold
1,2,Brad,-,Newton,taylorcatherine@example.net,(537) 674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver
2,3,Larry,-,Torres,dsanchez@example.net,(810) 256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze
3,4,Kimberly,-,Price,jessicaknight@example.com,(423) 222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold
4,5,Matthew,-,Phillips,qwilliams@example.com,(220) 763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold


2. Country Code Extraction and Mobile Number Enrichment
Task:
Extract the country code from the Address field.
Enrichment:
Map the extracted country code to its respective international phone dialing code.
Prepend this dialing code to the customer's Mobile Number to form a complete, internationally formatted phone number.

In [22]:
# Step 1: Extract region/country code (assuming format like "City, ST 12345" or "City, CountryCode ZIP")
def extract_region_code(address):
    match = re.search(r'\b([A-Z]{2})\s+\d{5}\b', address)
    return match.group(1) if match else None


In [26]:
customers.columns

Index(['customer_id', 'email', 'phone', 'address', 'registration_date',
       'loyalty_status', 'first_name', 'last_name', 'middle_name'],
      dtype='object')

In [63]:
customers['region_code'] = customers['address'].apply(extract_region_code)

In [64]:
# Step 2: Show unique region codes found
unique_codes = customers['region_code'].dropna().unique()
print("Unique region/country codes found in address:")
print(unique_codes)

Unique region/country codes found in address:
['AE' 'MT' 'UT' 'CA' 'OR' 'MA' 'CT' 'NC' 'WA' 'IL' 'WI' 'AA' 'FM' 'CO'
 'AP' 'RI' 'PW' 'FL' 'IA' 'VA' 'KS' 'SD' 'MP' 'AL' 'MH' 'TX' 'MS' 'MD'
 'NV' 'HI' 'GU' 'GA' 'ND' 'TN' 'ID' 'IN' 'DE' 'KY' 'NJ' 'AZ' 'MO' 'AS'
 'OH' 'OK' 'NE' 'NY' 'DC' 'ME' 'PA' 'WY' 'VI' 'SC' 'AR' 'NH' 'PR' 'WV'
 'AK' 'NM' 'MN' 'VT' 'MI' 'LA']


In [65]:
region_to_dial_code = {
    'AE': '+971',  # United Arab Emirates
    'MT': '+356',  # Malta
    'UT': '+1',    # Utah, USA
    'CA': '+1',    # California, USA
    'OR': '+1',    # Oregon, USA
    'MA': '+1',    # Massachusetts, USA
    'CT': '+1',    # Connecticut, USA
    'NC': '+1',    # North Carolina, USA
    'WA': '+1',    # Washington, USA
    'IL': '+1',    # Illinois, USA
    'WI': '+1',    # Wisconsin, USA
    'AA': '+1',    # Armed Forces Americas (USA)
    'FM': '+691',  # Micronesia
    'CO': '+1',    # Colorado, USA
    'AP': '+1',    # Armed Forces Pacific
    'RI': '+1',    # Rhode Island, USA
    'PW': '+680',  # Palau
    'FL': '+1',    # Florida, USA
    'IA': '+1',    # Iowa, USA
    'VA': '+1',    # Virginia, USA
    'KS': '+1',    # Kansas, USA
    'SD': '+1',    # South Dakota, USA
    'MP': '+1-670',# Northern Mariana Islands
    'AL': '+1',    # Alabama, USA
    'MH': '+692',  # Marshall Islands
    'TX': '+1',    # Texas, USA
    'MS': '+1',    # Mississippi, USA
    'MD': '+1',    # Maryland, USA
    'NV': '+1',    # Nevada, USA
    'HI': '+1',    # Hawaii, USA
    'GU': '+1-671',# Guam
    'GA': '+1',    # Georgia, USA
    'ND': '+1',    # North Dakota, USA
    'TN': '+1',    # Tennessee, USA
    'ID': '+1',    # Idaho, USA
    'IN': '+91',   # India
    'DE': '+49',   # Germany
    'KY': '+1',    # Kentucky, USA
    'NJ': '+1',    # New Jersey, USA
    'AZ': '+1',    # Arizona, USA
    'MO': '+1',    # Missouri, USA
    'AS': '+1',    # American Samoa
    'OH': '+1',    # Ohio, USA
    'OK': '+1',    # Oklahoma, USA
    'NE': '+1',    # Nebraska, USA
    'NY': '+1',    # New York, USA
    'DC': '+1',    # District of Columbia (Washington, D.C.)
    'ME': '+1',    # Maine, USA
    'PA': '+1',    # Pennsylvania, USA
    'WY': '+1',    # Wyoming, USA
    'VI': '+1-340',# U.S. Virgin Islands
    'SC': '+1',    # South Carolina, USA
    'AR': '+1',    # Arkansas, USA
    'NH': '+1',    # New Hampshire, USA
    'PR': '+1-787',# Puerto Rico
    'WV': '+1',    # West Virginia, USA
    'AK': '+1',    # Alaska, USA
    'NM': '+1',    # New Mexico, USA
    'MN': '+1',    # Minnesota, USA
    'VT': '+1',    # Vermont, USA
    'MI': '+1',    # Michigan, USA
    'LA': '+1',    # Louisiana, USA
}

In [66]:
dialing_codes = {
    'AE': '+971', 'MT': '+356', 'UT': '+1', 'CA': '+1', 'OR': '+1', 'MA': '+1',
    'CT': '+1', 'NC': '+1', 'WA': '+1', 'IL': '+1', 'WI': '+1', 'AA': '+1',
    'FM': '+691', 'CO': '+1', 'AP': '+1', 'RI': '+1', 'PW': '+680', 'FL': '+1',
    'IA': '+1', 'VA': '+1', 'KS': '+1', 'SD': '+1', 'MP': '+1-670', 'AL': '+1',
    'MH': '+692', 'TX': '+1', 'MS': '+1', 'MD': '+1', 'NV': '+1', 'HI': '+1',
    'GU': '+1-671', 'GA': '+1', 'ND': '+1', 'TN': '+1', 'ID': '+1', 'IN': '+1',
    'DE': '+1', 'KY': '+1', 'NJ': '+1', 'AZ': '+1', 'MO': '+1', 'AS': '+1-684',
    'OH': '+1', 'OK': '+1', 'NE': '+1', 'NY': '+1', 'DC': '+1', 'ME': '+1',
    'PA': '+1', 'WY': '+1', 'VI': '+1-340', 'SC': '+1', 'AR': '+1', 'NH': '+1',
    'PR': '+1-787', 'WV': '+1', 'AK': '+1', 'NM': '+1', 'MN': '+1', 'VT': '+1',
    'MI': '+1', 'LA': '+1'
}

In [67]:
customers["phone"] = (
    customers["address"].str.extract(r'([A-Z]{2})\s*$')[0]          # extract region code
    .map(region_to_dial_code)                                # map to dial code
    .fillna('') + customers["phone"].astype(str)                    # prepend dial code to phone
)


In [71]:
print(customers[["customer_id", "phone"]].head())

   customer_id           phone
0            1  (619) 723-4258
1            2  (537) 674-1158
2            3  (810) 256-4505
3            4  (423) 222-9779
4            5  (220) 763-3522


In [116]:
print(df['address'].head(10))

0                          USNS Santiago, FPO AE 80872
1    38783 Oliver Street, West Kristenborough, MT 9...
2       6845 Steele Turnpike, West Erikabury, UT 37487
3           1631 Alexis Meadows, Lake Amanda, CA 75179
4    2274 Williams Heights Suite 895, Andersonhaven...
5          19887 Brandon Bridge, New Rebecca, MA 94947
6          02590 Marshall Well, Sheppardland, CT 88067
7       209 Steven Oval Apt. 022, Mollyhaven, NC 10211
8             60123 Davis Ford, Mistychester, WA 69400
9            6182 Brown Mountain, South Gary, IL 29016
Name: address, dtype: object


In [72]:
# Extract 2-letter region code from the end of the address
customers["region_code"] = customers["address"].str.extract(r'([A-Z]{2})\s*\d*$')[0]

# Map region code to international dialing code
customers["dial_code"] = customers["region_code"].map(region_to_dial_code).fillna('')

# Prepend the dial code to the phone number
customers["phone"] = customers["dial_code"] + customers["phone"].astype(str)

# Drop intermediate columns if not needed
customers.drop(columns=["region_code", "dial_code"], inplace=True)

# Done!
print(customers[["customer_id", "phone"]].head())

   customer_id               phone
0            1  +971(619) 723-4258
1            2  +356(537) 674-1158
2            3    +1(810) 256-4505
3            4    +1(423) 222-9779
4            5    +1(220) 763-3522


In [73]:
import re

def clean_and_format_phone(row):
    phone = str(row['phone'])
    code = row['dialing_code']

    if pd.isnull(phone) or pd.isnull(code):
        return phone

    # Remove all non-digit characters
    phone_digits = re.sub(r'\D', '', phone)
    code_digits = re.sub(r'\D', '', code)

    # If the phone number already starts with the code, strip it
    if phone_digits.startswith(code_digits):
        phone_digits = phone_digits[len(code_digits):]

    # Use last 10 digits to ensure formatting is applied to base number
    phone_digits = phone_digits[-10:]

    # Format if we have exactly 10 digits
    if len(phone_digits) == 10:
        formatted = f"{code} ({phone_digits[:3]}) {phone_digits[3:6]}-{phone_digits[6:]}"
    else:
        # fallback: just add code and whatever digits we have
        formatted = f"{code} {phone_digits}"

    return formatted


In [75]:
print(customers[["customer_id", "phone"]].head())

   customer_id               phone
0            1  +971(619) 723-4258
1            2  +356(537) 674-1158
2            3    +1(810) 256-4505
3            4    +1(423) 222-9779
4            5    +1(220) 763-3522


In [79]:
# Run only if 'dialing_code' hasn't already been applied
def is_already_formatted(phone):
    return bool(re.match(r"^\+\d{1,4} \(\d{3}\) \d{3}-\d{4}$", str(phone)))

# Only format unprocessed numbers
customers['phone'] = customers.apply(
    lambda row: clean_and_format_phone(row) if not is_already_formatted(row['phone']) else row['phone'],
    axis=1
)


KeyError: 'dialing_code'

3. Customer Classification
Task:
Add a new column named Customer_Tier based on the following mappings:
Tier	
Mapping Value

Gold	2
Silver	1
Bronze	0

Tier	
Mapping Value

Gold	2
Silver	1
Bronze	0

In [36]:
print(customers.columns.tolist())


['customer_id', 'email', 'phone', 'address', 'registration_date', 'loyalty_status', 'first_name', 'last_name', 'middle_name', 'Customer_Tier']


In [52]:
from sqlalchemy import text

with engine.begin() as conn:
    result = conn.execute(text("SELECT DB_NAME()"))
    print(result.scalar())


sqlpythondb


In [31]:
customers['Customer_Tier'] = customers['Customer_Tier'].fillna('Bronze')


In [32]:
print(customers[customers['Customer_Tier'].isna()])


Empty DataFrame
Columns: [customer_id, email, phone, address, registration_date, loyalty_status, first_name, last_name, middle_name, Customer_Tier]
Index: []


In [37]:
import pandas as pd

# Replace 'customers' with your actual table name if different
query = "SELECT * FROM customers"

# Fetch the data
df = pd.read_sql(query, con=engine)

# Display the result
df


,customer_id,email,phone,address,registration_date,loyalty_status,first_name,last_name,middle_name
0,1,vayala@example.net,(619) 723-4258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold,Michelle,Kidd,-
1,2,taylorcatherine@example.net,(537) 674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver,Brad,Newton,-
2,3,dsanchez@example.net,(810) 256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze,Larry,Torres,-
3,4,jessicaknight@example.com,(423) 222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold,Kimberly,Price,-
4,5,qwilliams@example.com,(220) 763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold,Matthew,Phillips,-
...,...,...,...,...,...,...,...,...,...
995,996,walkerlisa@example.net,(638) 989-9441,"34746 Smith Gateway, New Sarah, AS 12715",2025-02-10,Silver,Jerry,Mcdaniel,-
996,997,eric24@example.org,(483) 625-2940,"2876 Tucker Road Suite 947, North Tommyborough...",2024-04-18,Bronze,Jodi,Simpson,-
997,998,pshaffer@example.net,(390) 747-3088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",2024-08-30,Bronze,Crystal,Brown,-
998,999,caitlindunlap@example.org,(257) 409-8196,"Unit 6377 Box 7662, DPO AP 03300",2024-05-16,Gold,Gregory,Duarte,-


In [ ]:
from sqlalchemy import text

# 1. Mapping loyalty_status to numeric tier
tier_mapping = {
    'Gold': 2,
    'Silver': 1,
    'Bronze': 0
}
customers['Customer_Tier'] = customers['loyalty_status'].map(tier_mapping)


In [ ]:
# 2. Add the column to the database table if not exists
with engine.begin() as conn:
    try:
        conn.execute(text("ALTER TABLE customers ADD Customer_Tier INT"))
    except Exception as e:
        print("Column may already exist, skipping ALTER TABLE:", e)

In [ ]:
# 3. Update the table with mapped values
with engine.begin() as conn:
    for _, row in customers.iterrows():
        if pd.notnull(row['Customer_Tier']):
            conn.execute(
                text("UPDATE customers SET Customer_Tier = :tier WHERE customer_id = :cid"),
                {"tier": int(row['Customer_Tier']), "cid": row['customer_id']}
            )

In [53]:
# Step 2: Read the updated table with the new column
customers = pd.read_sql("SELECT * FROM customers", engine)

# Optional: Print to verify
print(customers.columns.tolist())
customers.head()


['customer_id', 'email', 'phone', 'address', 'registration_date', 'loyalty_status', 'first_name', 'last_name', 'middle_name', 'Customer_Tier']


,customer_id,email,phone,address,registration_date,loyalty_status,first_name,last_name,middle_name,Customer_Tier
0,1,vayala@example.net,(619) 723-4258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold,Michelle,Kidd,-,2
1,2,taylorcatherine@example.net,(537) 674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver,Brad,Newton,-,1
2,3,dsanchez@example.net,(810) 256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze,Larry,Torres,-,0
3,4,jessicaknight@example.com,(423) 222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold,Kimberly,Price,-,2
4,5,qwilliams@example.com,(220) 763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold,Matthew,Phillips,-,2
